# Linear Regression Example

Let's walk through the steps of the official documentation example. Doing this will help your ability to read from the documentation, understand it, and then apply it to your own problems (the upcoming Consulting Project).

## Sample Data

```
-9.490009878824548 1:0.4551273600657362 2:0.36644694351969087 3:-0.38256108933468047 4:-0.4458430198517267 5:0.33109790358914726 6:0.8067445293443565 7:-0.2624341731773887 8:-0.44850386111659524 9:-0.07269284838169332 10:0.5658035575800715
0.2577820163584905 1:0.8386555657374337 2:-0.1270180511534269 3:0.499812362510895 4:-0.22686625128130267 5:-0.6452430441812433 6:0.18869982177936828 7:-0.5804648622673358 8:0.651931743775642 9:-0.6555641246242951 10:0.17485476357259122
```


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("LinearRegression").getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
# Load training data
dir_input_data_path = "./input_data/"
file_input_path = dir_input_data_path + "sample_linear_regression_data.txt"

df = spark.read.format("libsvm").load(file_input_path)

In [4]:
df.show(2)

+------------------+--------------------+
|             label|            features|
+------------------+--------------------+
|-9.490009878824548|(10,[0,1,2,3,4,5,...|
|0.2577820163584905|(10,[0,1,2,3,4,5,...|
+------------------+--------------------+
only showing top 2 rows



In [5]:
df.head(1)

[Row(label=-9.490009878824548, features=SparseVector(10, {0: 0.4551, 1: 0.3664, 2: -0.3826, 3: -0.4458, 4: 0.3311, 5: 0.8067, 6: -0.2624, 7: -0.4485, 8: -0.0727, 9: 0.5658}))]

In [6]:
df.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



Interesting! We haven't seen libsvm formats before. In fact the aren't very popular when working with datasets in Python, but the Spark Documentation makes use of them a lot because of their formatting. Let's see what the training data looks like:

In [7]:
df.show(3)

+------------------+--------------------+
|             label|            features|
+------------------+--------------------+
|-9.490009878824548|(10,[0,1,2,3,4,5,...|
|0.2577820163584905|(10,[0,1,2,3,4,5,...|
|-4.438869807456516|(10,[0,1,2,3,4,5,...|
+------------------+--------------------+
only showing top 3 rows



This is the format that Spark expects. Two columns with the names "label" and "features". 

The "label" column then needs to have the numerical label, either a regression numerical value, or a numerical value that matches to a classification grouping. Later on we will talk about unsupervised learning algorithms that by their nature do not use or require a label.

The feature column has inside of it a vector of all the features that belong to that row. Usually what we end up doing is combining the various feature columns we have into a single 'features' column using the data transformations we've learned about.

Let's continue working through this simple example!

In [8]:
df.columns

['label', 'features']

In [9]:
# These are the default values for the featuresCol, labelCol, predictionCol

# You could also pass in additional parameters for regularization, do the reading 
# in ISLR to fully understand that, after that its just some simple parameter calls.
# Check the documentation with Shift+Tab for more info!
lr = LinearRegression(featuresCol="features", 
                      labelCol="label", 
                      predictionCol="prediction")

In [10]:
# Fit the model
lr_model = lr.fit(df)

In [11]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {}".format(str(lr_model.coefficients))) # For each feature...
print('\n')
print("Intercept:{}".format(str(lr_model.intercept)))

Coefficients: [0.0073350710225801715,0.8313757584337543,-0.8095307954684084,2.441191686884721,0.5191713795290003,1.1534591903547016,-0.2989124112808717,-0.5128514186201779,-0.619712827067017,0.6956151804322931]


Intercept:0.14228558260358093


In [12]:
lr_model.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [13]:
lr_model.intercept

0.14228558260358093

In [14]:
lr_model.loss

Param(parent='LinearRegression_f83e6de75176', name='loss', doc='The loss function to be optimized. Supported options: squaredError, huber. (Default squaredError)')

In [15]:
lr_model.scale

1.0

There is a summary attribute that contains even more info!

In [16]:
# Summarize the model over the training set and print out some metrics
trainingSummary = lr_model.summary

Lots of info, here are a few examples:

In [17]:
trainingSummary.residuals.show()
print("RMSE: {}".format(trainingSummary.rootMeanSquaredError))
print("r2: {}".format(trainingSummary.r2))

+-------------------+
|          residuals|
+-------------------+
|-11.011130022096554|
| 0.9236590911176538|
|-4.5957401897776675|
|  -20.4201774575836|
|-10.339160314788181|
|-5.9552091439610555|
|-10.726906349283922|
|  2.122807193191233|
|  4.077122222293811|
|-17.316168071241652|
| -4.593044343959059|
|  6.380476690746936|
| 11.320566035059846|
|-20.721971774534094|
| -2.736692773777401|
| -16.66886934252847|
|  8.242186378876315|
|-1.3723486332690233|
|-0.7060332131264666|
|-1.1591135969994064|
+-------------------+
only showing top 20 rows

RMSE: 10.16309157133015
r2: 0.027839179518600154


In [18]:
lr_model_summary = lr_model.summary

In [19]:
print(lr_model_summary.coefficientStandardErrors)

[0.8068799897284031, 0.817504726374364, 0.8095690515350549, 0.8435177715251213, 0.8066923009911938, 0.8238680228428261, 0.8041910472918519, 0.8095101717564966, 0.8428997032677101, 0.788760166505627, 0.46405794834415603]


In [20]:
lr_model_summary.meanAbsoluteError

8.145215527783876

In [21]:
lr_model_summary.meanSquaredError

103.28843028724194

In [22]:
lr_model_summary.rootMeanSquaredError

10.16309157133015

## Train/Test Splits

But wait! We've commited a big mistake, we never separated our data set into a training and test set. Instead we trained on ALL of the data, something we generally want to avoid doing. Read ISLR and check out the theory lecture for more info on this, but remember we won't get a fair evaluation of our model by judging how well it does again on the same data it was trained on!

Luckily Spark DataFrames have an almost too convienent method of splitting the data! Let's see it:

In [23]:
# Load all corpus

In [24]:
df = spark.read.format("libsvm").load(file_input_path)

In [25]:
df.head(2)

[Row(label=-9.490009878824548, features=SparseVector(10, {0: 0.4551, 1: 0.3664, 2: -0.3826, 3: -0.4458, 4: 0.3311, 5: 0.8067, 6: -0.2624, 7: -0.4485, 8: -0.0727, 9: 0.5658})),
 Row(label=0.2577820163584905, features=SparseVector(10, {0: 0.8387, 1: -0.127, 2: 0.4998, 3: -0.2269, 4: -0.6452, 5: 0.1887, 6: -0.5805, 7: 0.6519, 8: -0.6556, 9: 0.1749}))]

In [26]:
df.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [27]:
df.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                501|
|   mean|0.25688882219498976|
| stddev| 10.317884030544564|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [28]:
# Pass in the split between training/test as a list.
# No correct, but generally 70/30 or 60/40 splits are used. 
# Depending on how much data you have and how unbalanced it is.

In [29]:
train_set, test_set = df.randomSplit([0.8, 0.2])

In [30]:
# Review training set and testing set

In [31]:
train_set.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                404|
|   mean|0.27941508411735216|
| stddev| 10.492148919662801|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [32]:
test_set.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                 97|
|   mean|0.16306810243586653|
| stddev|  9.609010234318017|
|    min|-19.782762789614537|
|    max| 26.903524792043335|
+-------+-------------------+



Now we only train on the train_data

In [33]:
train_set.columns

['label', 'features']

## Train and Fit Linear Regression Model

In [34]:
from pyspark.ml.regression import LinearRegression

# Create Model
lr = LinearRegression(featuresCol="features", labelCol="label")

# Fit Model
lr_model = lr.fit(train_set)

Now we can directly get a .summary object using the evaluate method and Show the RMSE score

In [35]:
lr_model_summary = lr_model.summary

In [36]:
prediction_result = lr_model.evaluate(test_set)

## Show Residuals of Prediction Results

In [37]:
prediction_result.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -20.77398363324071|
|-17.232412233213406|
| -15.94614835086861|
|-15.579687029130524|
|-16.120564632791236|
|-17.234413337015194|
|-13.461913671444306|
|-12.393319399015537|
| -13.76524027667394|
|-10.703912152503811|
| -7.704014094082844|
| -10.40938185408925|
|  -8.49544563110033|
| -8.942255084713626|
| -9.973151856288984|
|-11.013166825397336|
| -5.632369222707707|
| -9.895470377407369|
| -9.208015640940916|
| -9.590107983250439|
+-------------------+
only showing top 20 rows



## Show the RMSE of Training set and Testing set

In [38]:
print("RMSE of training set : {}".format(lr_model_summary.rootMeanSquaredError))
print("RMSE of testing set: {}".format(prediction_result.rootMeanSquaredError))

RMSE of training set : 10.365455688124422
RMSE of testing set: 9.359380490785071


## Get Prediction Values

Well that is nice, but realistically we will eventually want to test this model against unlabeled data, after all, that is the whole point of building the model in the first place. We can again do this with a convenient method call, in this case, transform(). Which was actually being called within the evaluate() method. Let's see it in action:

In [39]:
unlabeled_set = test_set.select("features")

In [40]:
unlabeled_set.show(2)

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 2 rows



In [41]:
unlabeled_set.head(2)

[Row(features=SparseVector(10, {0: -0.0389, 1: -0.4167, 2: 0.8997, 3: 0.641, 4: 0.2733, 5: -0.2618, 6: -0.2795, 7: -0.1307, 8: -0.0854, 9: -0.0546})),
 Row(features=SparseVector(10, {0: 0.5176, 1: 0.2305, 2: 0.6163, 3: 0.1691, 4: 0.9695, 5: -0.3471, 6: 0.8527, 7: 0.9896, 8: 0.8806, 9: -0.4345}))]

In [42]:
pred_result = lr_model.transform(unlabeled_set)

In [43]:
pred_result.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...| 0.9912208436261734|
|(10,[0,1,2,3,4,5,...| -0.571213955451109|
|(10,[0,1,2,3,4,5,...|-1.0803439133409376|
|(10,[0,1,2,3,4,5,...|-0.5059720118909656|
|(10,[0,1,2,3,4,5,...|0.25855530522067516|
|(10,[0,1,2,3,4,5,...| 1.5023250647759474|
|(10,[0,1,2,3,4,5,...|-0.4051742237144631|
|(10,[0,1,2,3,4,5,...|-1.0272753768752207|
|(10,[0,1,2,3,4,5,...|   1.20666448781775|
|(10,[0,1,2,3,4,5,...|-1.1534382129256144|
|(10,[0,1,2,3,4,5,...| -3.936535583805983|
|(10,[0,1,2,3,4,5,...|0.11371591805933007|
|(10,[0,1,2,3,4,5,...| -1.737993955852824|
|(10,[0,1,2,3,4,5,...|-0.9499008431125953|
|(10,[0,1,2,3,4,5,...|0.20265130994342195|
|(10,[0,1,2,3,4,5,...|  1.523156946572788|
|(10,[0,1,2,3,4,5,...|-3.7966187088062338|
|(10,[0,1,2,3,4,5,...| 0.6844038063251215|
|(10,[0,1,2,3,4,5,...|0.21595999999264612|
|(10,[0,1,2,3,4,5,...| 0.9843944684883473|
+----------

Okay, so this data is a bit meaningless, so let's explore this same process with some data that actually makes a little more intuitive sense!